In [38]:
from bs4 import BeautifulSoup
import requests
import re

page_content = requests.get("https://www.instagram.com/mkbhd/").content
soup = BeautifulSoup(page_content, 'html.parser')

metas = soup.find_all('meta')
description = [ meta.attrs['content'] for meta in metas if 'name' in meta.attrs and meta.attrs['name'] == 'description' ][0]
numbers = re.findall(r'\d+', description)
int(description.split('-')[0].strip().split(' ')[-2].replace(',',''))

1042

In [45]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import csv
import json


def get_handle(url):
    splits = url.split('/')
    if url:
        if url.endswith('/'):
            handle = splits[-2]
        else:
            handle = splits[-1]
    else:
        handle = None    
    return handle
        
columns = ['tw_handle','ig_handle','ig_posts','ig_thumbnail_url']
outdf = pd.DataFrame(columns=columns)

print(outdf)
df = pd.read_csv('influencer_list.csv')
for index, row in df.iterrows():
    influencer = row['Influencer']
    category = row['Category']
    twitter_url = row['Twitter']
    twitter_handle = get_handle(twitter_url)
    
    instagram_url = row['Instagram']

    if instagram_url != 'null':
        instagram_handle = get_handle(instagram_url)
        page_content = requests.get(instagram_url).content
        soup = BeautifulSoup(page_content, 'html.parser')

        metas = soup.find_all('meta')
        description = [ meta.attrs['content'] for meta in metas if 'name' in meta.attrs and meta.attrs['name'] == 'description' ][0]
        numbers = re.findall(r'\d+', description)
        num_posts = description.split('-')[0].strip().split(' ')[-2].replace(',','')
        thumbnail_url = soup.find("meta",  property="og:image")['content']
    else:
        num_posts = 0
    result_list = [twitter_handle, instagram_handle, num_posts, thumbnail_url]
    outdf.loc[index] = result_list

outdf.to_csv('instagram_posts_details.csv', sep=';', index=False)



Empty DataFrame
Columns: [tw_handle, ig_handle, ig_posts, ig_thumbnail_url]
Index: []


In [48]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client.influencers_db

path = '/Users/manojkarthick/Documents/Spring-18/CMPT-733/Project-733/instagram_posts_details.csv'
igdf = pd.read_csv(path, sep=';')

collection = db.instagram_posts_collection

records = igdf.to_dict(orient='records')
op = collection.insert_many(records)
print(op)